In [1]:
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
# Create engine
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [3]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [4]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
# Find table columns
Station.__table__.columns.keys()

['id', 'station', 'name', 'latitude', 'longitude', 'elevation']

In [6]:
# Find table columns
Measurement.__table__.columns.keys()

['id', 'station', 'date', 'prcp', 'tobs']

In [6]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [15]:
# Define trip dates
start_date = '2017-02-01'
end_date = '2017-02-08'

## Query for precipitation

In [65]:
# Create precipitation dictionary {'date':'prcp'} with average precipitation from each station
prcp_query = session.query(Measurement.date, func.avg(Measurement.prcp).label('prcp')).group_by(Measurement.date)

prcp_list = []

# Add rows from query to dict
for row in prcp_query:
    prcp_list.append(row._asdict())

In [67]:
prcp_list

[{'date': '2010-01-01', 'prcp': 0.15142857142857144},
 {'date': '2010-01-02', 'prcp': 0.002857142857142857},
 {'date': '2010-01-03', 'prcp': 0.0},
 {'date': '2010-01-04', 'prcp': 0.0014285714285714286},
 {'date': '2010-01-05', 'prcp': 0.005},
 {'date': '2010-01-06', 'prcp': 0.11600000000000002},
 {'date': '2010-01-07', 'prcp': 0.27499999999999997},
 {'date': '2010-01-08', 'prcp': 0.008571428571428572},
 {'date': '2010-01-09', 'prcp': 0.002857142857142857},
 {'date': '2010-01-10', 'prcp': 0.002857142857142857},
 {'date': '2010-01-11', 'prcp': 0.12571428571428572},
 {'date': '2010-01-12', 'prcp': 0.002857142857142857},
 {'date': '2010-01-13', 'prcp': 0.0033333333333333335},
 {'date': '2010-01-14', 'prcp': 0.0},
 {'date': '2010-01-15', 'prcp': 0.0},
 {'date': '2010-01-16', 'prcp': 0.0},
 {'date': '2010-01-17', 'prcp': 0.0},
 {'date': '2010-01-18', 'prcp': 0.0},
 {'date': '2010-01-19', 'prcp': 0.0},
 {'date': '2010-01-20', 'prcp': 0.0014285714285714286},
 {'date': '2010-01-21', 'prcp': 0.1

## Query for stations

In [71]:
# Create stations dictionary 
stations_query = session.query(Station.name).group_by(Station.name)

stations_list = []

# Add rows from query to dict
for row in stations_query:
    stations_list.append(row._asdict())

In [72]:
stations_list

[{'name': 'HONOLULU OBSERVATORY 702.2, HI US'},
 {'name': 'KANEOHE 838.1, HI US'},
 {'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US'},
 {'name': 'MANOA LYON ARBO 785.2, HI US'},
 {'name': 'PEARL CITY, HI US'},
 {'name': 'UPPER WAHIAWA 874.3, HI US'},
 {'name': 'WAIHEE 837.5, HI US'},
 {'name': 'WAIKIKI 717.2, HI US'},
 {'name': 'WAIMANALO EXPERIMENTAL FARM, HI US'}]

## Query for tobs

In [76]:
# Find most recent date in the data
session.query(Measurement.date, Measurement.prcp).order_by(Measurement.date.desc()).limit(1).all()

[('2017-08-23', 0.0)]

In [77]:
tobs_query = session.query(Measurement.date, func.avg(Measurement.tobs).label('tobs'))\
    .filter(Measurement.date >= '2016-08-23').group_by(Measurement.date)

tobs_list = []

# Add rows from query to dict
for row in tobs_query:
    tobs_list.append(row._asdict())

In [78]:
tobs_list

[{'date': '2016-08-23', 'tobs': 78.28571428571429},
 {'date': '2016-08-24', 'tobs': 77.57142857142857},
 {'date': '2016-08-25', 'tobs': 79.42857142857143},
 {'date': '2016-08-26', 'tobs': 80.5},
 {'date': '2016-08-27', 'tobs': 76.83333333333333},
 {'date': '2016-08-28', 'tobs': 76.0},
 {'date': '2016-08-29', 'tobs': 78.71428571428571},
 {'date': '2016-08-30', 'tobs': 77.33333333333333},
 {'date': '2016-08-31', 'tobs': 78.28571428571429},
 {'date': '2016-09-01', 'tobs': 80.16666666666667},
 {'date': '2016-09-02', 'tobs': 79.42857142857143},
 {'date': '2016-09-03', 'tobs': 77.4},
 {'date': '2016-09-04', 'tobs': 77.0},
 {'date': '2016-09-05', 'tobs': 79.5},
 {'date': '2016-09-06', 'tobs': 75.66666666666667},
 {'date': '2016-09-07', 'tobs': 75.83333333333333},
 {'date': '2016-09-08', 'tobs': 78.71428571428571},
 {'date': '2016-09-09', 'tobs': 77.42857142857143},
 {'date': '2016-09-10', 'tobs': 76.8},
 {'date': '2016-09-11', 'tobs': 77.2},
 {'date': '2016-09-12', 'tobs': 76.85714285714286},

## Query for min, max, avg. temp

In [89]:
# Define trip dates
start_date = '2017-02-01'
end_date = '2017-02-08'

In [90]:
stats_query = session.query(func.avg(Measurement.tobs).label('average'), func.min(Measurement.tobs).label('min'), func.max(Measurement.tobs).label('max'))\
    .filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

stats_list = []

# Add rows from query to dict
for row in stats_query:
    stats_list.append(row._asdict())

In [91]:
stats_list

[{'average': 68.9423076923077, 'min': 59.0, 'max': 80.0}]